In [1]:
using LinearAlgebra
using JuMP
using GLPK
using MathOptInterface

In [2]:
#for ease of comprehension, I will use capital letters for variables outside the model space
#establish port quantities and market demands

P = [15,25,35,45]
M = [25,25,35,15,20]

#establish travel requirements
#for now, treat these values as B[M,N] = cost of transporting one unit of goods from P[M] to M[N]
#proposed improvement: pull actual distances and travel times from Google API...
#https://developers.google.com/maps/documentation/distance-matrix/intro

B = [
    1 7 5 3 9;
    3 5 7 9 1;
    5 3 9 1 7;
    7 9 1 3 5
]

#collect dimensionality of matrix B for later use
Ports, Markets = size(B)

#create iterable arrays that represent the dimensionality of matrix B
#I = the number of markets
I = collect(1:Markets)
#J = the number of ports
J = collect(1:Ports)

4-element Array{Int64,1}:
 1
 2
 3
 4

In [3]:
function NorthwestRowFill(supply, demand, output, row)
    #extract dimensionality of output array
    num_row, num_col = size(output)
    println(num_row, num_col)
    #iterate over columns within supplied row value
    for c in 1:num_col
        if supply[row] == 0
            break
        #if supply <= demand, put all supply towards that demand
        elseif supply[row] <= demand[c]
            #push supply into output matrix
            #println("setting output (", row,",", c, ") to ", supply[row])
            output[row,c] = supply[row]
            #reduce demand at that market
            demand[c] = demand[c] - supply[row]
            #zero supply from that port
            supply[row] = 0
        #if supply > demand, fill all demand and continue across columns    
        else  
            output[row,c] = demand[c]
            supply[row] = supply[row] - demand[c]
            demand[c] = 0
        end
    end
    return output
end

NorthwestRowFill (generic function with 1 method)

In [4]:
function BalancedFeasibility(supply::Array , demand::Array, cost:: Array)
    #extract dimensionality of costs matrix
    rows, cols = size(cost)
    #check that there are the same number of rows in costs matrix and ports array
    if size(supply)[1] != rows
        throw(DimensionMismatch("Ports array does not match rows of cost matrix"))
    #check that there are the same number of columns in costs matrix as markets array
    elseif size(demand)[1] != cols
        throw(DimensionMismatch("Markets array does not match columns of cost matrix"))
    #check that demand and supply are equal   
    elseif sum(supply) != sum(demand)
        throw(DomainError("The sum of available products and product demand is not equal"))
    end
    #print("Dimensions matched and market is balanced.  Continuing...")
end

BalancedFeasibility (generic function with 1 method)

In [5]:
function NorthwestCorner(supply::Array , demand::Array, cost:: Array)
    #copy input supply and demand arrays
    s = copy(supply)
    d = copy(demand)
    #feasibility check
    BalancedFeasibility(supply, demand, cost)
    #intialize output matrix
    rows, cols = size(cost)
    output = zeros(rows, cols)
    for r in 1:rows
        output = NorthwestRowFill(s, d, output, r)
    end
    display(output)
    return output
end

NorthwestCorner (generic function with 1 method)

In [6]:
feasible_solution = NorthwestCorner(P,M,B)

4×5 Array{Float64,2}:
 15.0   0.0   0.0   0.0   0.0
 10.0  15.0   0.0   0.0   0.0
  0.0  10.0  25.0   0.0   0.0
  0.0   0.0  10.0  15.0  20.0

45
45
45
45


4×5 Array{Float64,2}:
 15.0   0.0   0.0   0.0   0.0
 10.0  15.0   0.0   0.0   0.0
  0.0  10.0  25.0   0.0   0.0
  0.0   0.0  10.0  15.0  20.0

In [7]:
function LeastCostIterator(s, d, c, output)
    num_row, num_col = size(c)
    for row in 1:num_row
        #println("on row $row")
        if s[row] == 0
            continue
        else
            lowest_cost_loc = CartesianIndex(row, findmin(c[row,:])[2])
            cheap_y = lowest_cost_loc[2]
            #println("y index of lowest cost is $cheap_y")
            #println("found lowest cost at $lowest_cost_loc")
            if s[row] >= d[cheap_y]
                output[lowest_cost_loc] = d[cheap_y]
                s[row] = s[row] - d[cheap_y]
                d[cheap_y] = 0
                c[lowest_cost_loc] = 1000000
                #if supply <= demand, put all supply towards that demand
            else 
                output[lowest_cost_loc] = s[row]
                    #reduce demand at that market
                d[cheap_y] = d[cheap_y] - s[row]
                    #zero supply from that port
                s[row] = 0
                c[lowest_cost_loc] = 1000000
            end                  
        end
    end
    return s, d, c, output
end

LeastCostIterator (generic function with 1 method)

In [8]:
function LeastCostRule(supply, demand, cost)
    num_row, num_col = size(cost)
    s = copy(supply)
    d = copy(demand)
    c = copy(cost)
    output = zeros(num_row, num_col)
    for repeats in 1:num_col 
        s, d, c, output = LeastCostIterator(s,d,c, output)
    end
    display(output)
    return output
end

LeastCostRule (generic function with 1 method)

In [9]:
least_output = LeastCostRule(P,M,B)

4×5 Array{Float64,2}:
 15.0   0.0   0.0   0.0   0.0
  5.0   0.0   0.0   0.0  20.0
  0.0  20.0   0.0  15.0   0.0
  5.0   5.0  35.0   0.0   0.0

4×5 Array{Float64,2}:
 15.0   0.0   0.0   0.0   0.0
  5.0   0.0   0.0   0.0  20.0
  0.0  20.0   0.0  15.0   0.0
  5.0   5.0  35.0   0.0   0.0

In [10]:
u, v = ComputeUV(least_output, B)
is_opt, w, loc_max = ComputeW(u,v,least_output, B)

UndefVarError: UndefVarError: ComputeUV not defined

In [11]:
function TransportationSimplexOpt(supply, demand, cost)
    println("Finding Northwest Corner feasible solution...")
    nwc_feasible_solution = NorthwestCorner(supply, demand, cost)
    
    println("Finding Least Cost feasible solution...")
    lc_feasible_solution = LeastCostRule(supply, demand, cost)

    println("Attempting to Optimize NWC feasible solution")
    nwc_u,nwc_v = ComputeUV(nwc_feasible_solution, cost)
    println("Found u of $nwc_u")
    println("Found v of $nwc_v")
    nwc_is_not_opt, nwc_w, nwc_loc_max = ComputeW(nwc_u, nwc_v, nwc_feasible_solution, cost)
    if nwc_is_not_opt
        println("NWC solution is not optimal.  Looping...")
    else
        println("Congrats!  You have an optimal solution.")
        return nwc_feasible_solution
    end
    nwc_first_steps = FindFirstStep(nwc_loc_max, nwc_feasible_solution)
    if FindLoop(nwc_loc_max, nwc_feasible_solution, nwc_first_steps, nwc_w) == false
        println("NWC solution is not loopable - falling back to Least Cost option")
        lc_u, lc_v = ComputeUV(lc_feasible_solution, cost)
        lc_is_not_opt, lc_w, lc_loc_max = ComputeW(lc_u, lc_v, lc_feasible_solution, cost)
                if lc_is_not_opt
                    println("LC solution is not optimal.  Looping...")
                else
                    println("Congrats!  You have an optimal solution.")
                    return nwc_feasible_solution
                end
        lc_first_steps = FindFirstStep(lc_loc_max, lc_feasible_solution)
        loop = FindLoop(lc_loc_max, lc_feasible_solution, lc_first_steps, lc_w)
        feasible_solution = lc_feasible_solution
        is_not_opt = lc_is_not_opt
    else
        loop = FindLoop(nwc_loc_max, nwc_feasible_solution, nwc_first_steps, nwc_w)
        feasible_solution = nwc_feasible_solution
        is_not_opt = nwc_is_not_opt
    end
    
    feasible_solution = LoopTransform(loop, feasible_solution)
    iteration = 1
    while is_not_opt
        println("in while loop - finding u & v")
        u,v = ComputeUV(feasible_solution, cost)
        print(u)
        print(v)
        println("in while loop - finding w")
        is_not_opt, w, loc_max = ComputeW(u, v, feasible_solution, cost)
        print(w)
            if is_not_opt == false
                print("Congrats!  You found an optimal solution!")
                return feasible_solution
            else
                iteration += 1
                println("Value loop iteration $iteration")
            end
        first_steps = FindFirstStep(loc_max, feasible_solution)
        loop = FindLoop(loc_max, feasible_solution, first_steps, w)
        feasible_solution = LoopTransform(loop, feasible_solution)
    end
    return feasible_solution
        
end

TransportationSimplexOpt (generic function with 1 method)

In [23]:
manual_opt = TransportationSimplexOpt(P,M,B)

4×5 Array{Float64,2}:
 15.0   0.0   0.0   0.0   0.0
 10.0  15.0   0.0   0.0   0.0
  0.0  10.0  25.0   0.0   0.0
  0.0   0.0  10.0  15.0  20.0

4×5 Array{Float64,2}:
 15.0   0.0   0.0   0.0   0.0
  5.0   0.0   0.0   0.0  20.0
  0.0  20.0   0.0  15.0   0.0
  5.0   5.0  35.0   0.0   0.0

Finding Northwest Corner feasible solution...
45
45
45
45
Finding Least Cost feasible solution...
Attempting to Optimize NWC feasible solution
[0.0, NaN, NaN, NaN]Found u of [0.0, 2.0, 0.0, -8.0]
Found v of [1.0, 3.0, 9.0, 11.0, 13.0]
NWC solution is not optimal.  Looping...
checking (2, 1)
checking (2, 2)
checking (4, 5)
NWC solution is not loopable - falling back to Least Cost option
[0.0, NaN, NaN, NaN]LC solution is not optimal.  Looping...
checking (4, 1)
checking (4, 2)
Good news - I found a row loop!
moving 5.0 around the loop
in while loop - finding u & v
[0.0, NaN, NaN, NaN][0.0, 2.0, 0.0, 6.0][1.0, 3.0, -5.0, -3.0, -1.0]in while loop - finding w
[0.0 -4.0 -10.0 -6.0 -10.0; 0.0 0.0 -10.0 -10.0 0.0; -4.0 0.0 -14.0 0.0 -8.0; 0.0 0.0 0.0 0.0 0.0]Congrats!  You found an optimal solution!

4×5 Array{Float64,2}:
 15.0   0.0   0.0   0.0   0.0
  5.0   0.0   0.0   0.0  20.0
  0.0  25.0   0.0  10.0   0.0
  5.0   0.0  35.0   5.0   0.0

In [24]:
sum(manual_opt .* B)

220.0

In [14]:
function ComputeUV(feasible_solution:: Array, costs:: Array)
    #extract dimensions, initialize output u and v arrays
    num_row, num_col = size(feasible_solution)
    u = fill(NaN, num_row)
    v = fill(NaN, num_col)
    #copy feasible solution, set u[1] = 0 by convention
    bfs = copy(feasible_solution)
    u[1] = 0
    print(u)
    for row in 1:num_row
        #println("on row $row")
        for col in 1:num_col
            #println("on col $col")
            if feasible_solution[row,col] == 0
                #println("no value at $row, $col - pass")
                continue
            elseif isnan(u[row])
                #println("setting u $row value")
                if isnan(v[col])
                    #println("override to set u $row to 0")
                    u[row] = 0
                    v[col] = costs[row,col] - u[row]
                else                
                    u[row] = costs[row,col] - v[col]
                    #println("u is now $u")
                end
            else
                #println("setting v $col value")
                v[col] = costs[row,col] - u[row]
                #println("v is now $v")
            end
        end
    end
    return u,v
end

ComputeUV (generic function with 1 method)

In [15]:
u, v = ComputeUV(feasible_solution, B)

[0.0, NaN, NaN, NaN]

([0.0, 2.0, 0.0, -8.0], [1.0, 3.0, 9.0, 11.0, 13.0])

In [16]:
function ComputeW(u,v,feasible_solution, costs)
    num_row, num_col = size(feasible_solution)
    w = fill(NaN, (num_row, num_col))
    for row in 1:num_row
        for col in 1:num_col
            if feasible_solution[row,col] != 0
                w[row,col] = 0
            else
                w[row,col] = u[row] + v[col] - costs[row,col]
            end
        end
    end
    if findmax(w)[1] <= 0
        return false, w, findmax(w)[2]
    else
        return true, w, findmax(w)[2]
    end
end

ComputeW (generic function with 1 method)

In [429]:
is_opt, w, loc_max = ComputeW(u,v,feasible_solution, B)

(true, [0.0 -4.0 … 8.0 4.0; 0.0 0.0 … 4.0 14.0; -4.0 0.0 … 10.0 6.0; -14.0 -14.0 … 0.0 0.0], CartesianIndex(2, 5))

In [17]:
sum(w .< 0)
index_list = 1:size(list)[1]
print(index_list)

UndefVarError: UndefVarError: w not defined

In [18]:
function FindFirstStep(starting_coordinate,feasible_solution)
    num_row, num_col = size(feasible_solution)
    starting_x = starting_coordinate[1]
    starting_y = starting_coordinate[2]
    move = []
    #println("$starting_x, $starting_y")
    for col in 1:num_col
        if feasible_solution[starting_x, col] > 0
            #println("found x at $starting_x, $col")
            push!(move, (starting_x, col))
        end
    end
    for row in 1:num_row
        if feasible_solution[row,starting_y] > 0
            #println("found y at $row, $starting_y")
            push!(move, (row, starting_y))
        end
    end
    return move
end

FindFirstStep (generic function with 1 method)

In [434]:
my_moves = FindFirstStep(loc_max, least_output)
my_moves

4, 4
found x at 4, 1
found x at 4, 2
found x at 4, 3
found y at 3, 4


4-element Array{Any,1}:
 (4, 1)
 (4, 2)
 (4, 3)
 (3, 4)

In [19]:
function FindLoop(starting_coordinate, feasible_solution, first_moves, w)
    output = []
    num_row, num_col = size(feasible_solution)
    starting_x = starting_coordinate[1]
    starting_y = starting_coordinate[2]
    for coord in first_moves
        println("checking $coord")
        this_x = coord[1]
        this_y = coord[2]
        for row in 1:num_row
            if row == this_x
                continue
            else
                if feasible_solution[row, this_y] > 0 && feasible_solution[row, starting_y] > 0
                    println("Good news - I found a row loop!")
                    push!(output, starting_coordinate, CartesianIndex(coord), CartesianIndex(row, this_y), CartesianIndex(row, starting_y))
                    return output
                else
                    #println("No loop-back at point $row, $this_y")
                    continue
                end
            end
        end
        for col in 1:num_col
            if col == this_y || this_x == starting_x
                continue
            else
                if feasible_solution[this_x, col] > 0 && feasible_solution[starting_x, col] > 0
                    println("Good news - I found a column loop!")
                    push!(output, starting_coordinate, CartesianIndex(coord), CartesianIndex(this_x, col), CartesianIndex(starting_x, col))
                    return output
                else
                    #println("No loop-back at point $this_x, $col")
                    continue
                end
            end
        end
            
    end
    return false
end

FindLoop (generic function with 1 method)

In [20]:
loops = FindLoop(loc_max, least_output, my_moves, w)

UndefVarError: UndefVarError: loc_max not defined

In [21]:
function LoopTransform(loops, feasible_solution)
    loop_values = []
    for value in loops
        push!(loop_values, feasible_solution[value])
    end
    value_to_move, ignore = findmin(loop_values[loop_values .> 0])
    println("moving $value_to_move around the loop")
    for (index, value) in enumerate(loops)
        if index % 2 != 0
            feasible_solution[value] = feasible_solution[value] + value_to_move
        else
            feasible_solution[value] = feasible_solution[value] - value_to_move
        end
    end 
    return feasible_solution
end

LoopTransform (generic function with 1 method)

In [22]:
LoopTransform(loops, least_output)

UndefVarError: UndefVarError: loops not defined

In [167]:
#initialize model
basic_model = Model(with_optimizer(GLPK.Optimizer))

#initialize variables
#for ease of comprehension, I will use lower-case letters for variables *inside* the model space
#x[i,j] = units moved from Market[i] to Port[j] 
@variable(basic_model, x[1:Ports,1:Markets] >= 0)

#P will represent units available in each port
@variable(basic_model, p[i=1:Ports], lower_bound = 0, upper_bound = P[i])

#M will represent units demanded in each market
@variable(basic_model, m[i=1:Markets], lower_bound = 0, upper_bound = M[i])

#create slack variables for excess demand and supply
#forced to zero for now, but implemented for future use
@variable(basic_model, slackdemand == 0)
@variable(basic_model, slacksupply == 0)

#add constraints
#demand constraints - column-wise sum of matrix x should equal market demand for a given M
for market in I
    @constraint(basic_model, sum(x[:,market]) + slackdemand == M[market])
end

#supply constraints - row-wise sum of matrix x should equual supply for a given P
for port in J
    @constraint(basic_model, sum(x[port,:]) + slacksupply == P[port]) 
end

#compose the objective function
@objective(basic_model, Min, sum(B .* x))

x[1,1] + 3 x[2,1] + 5 x[3,1] + 7 x[4,1] + 7 x[1,2] + 5 x[2,2] + 3 x[3,2] + 9 x[4,2] + 5 x[1,3] + 7 x[2,3] + 9 x[3,3] + x[4,3] + 3 x[1,4] + 9 x[2,4] + x[3,4] + 3 x[4,4] + 9 x[1,5] + x[2,5] + 7 x[3,5] + 5 x[4,5]

In [168]:
@time begin
    status = optimize!(basic_model)
end

 19.897766 seconds (27.91 M allocations: 1.373 GiB, 6.06% gc time)


In [169]:
#print objective value
println("Final transportation cost:", JuMP.objective_value(basic_model))

#print values of all variables in system
#first, extract list of all variables - no built-in for JuMP to dump all vars
list = JuMP.all_variables(basic_model)

#now iterate over list and print values
for name in list
    println(name,":", JuMP.value(name))
end

Final transportation cost:240.0
x[1,1]:15.0
x[1,2]:0.0
x[1,3]:0.0
x[1,4]:0.0
x[1,5]:0.0
x[2,1]:10.0
x[2,2]:0.0
x[2,3]:0.0
x[2,4]:0.0
x[2,5]:15.0
x[3,1]:0.0
x[3,2]:25.0
x[3,3]:0.0
x[3,4]:10.0
x[3,5]:0.0
x[4,1]:0.0
x[4,2]:0.0
x[4,3]:25.0
x[4,4]:15.0
x[4,5]:5.0
p[1]:0.0
p[2]:0.0
p[3]:0.0
p[4]:0.0
m[1]:0.0
m[2]:0.0
m[3]:0.0
m[4]:0.0
m[5]:0.0
slackdemand:0.0
slacksupply:0.0
